In [99]:
# Test Google API key and list available models
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✓ Google API Key loaded")
    print(f"Key starts with: {GOOGLE_API_KEY[:20]}...")
    
    # List available models
    print("\n📋 Available Gemini models:")
    for m in genai.list_models():
        if 'gemini' in m.name.lower():
            print(f"  - {m.name}")
else:
    print("❌ No Google API key found!")

✓ Google API Key loaded
Key starts with: AIzaSyBcbIPq_vyLqxhP...

📋 Available Gemini models:
  - models/gemini-2.5-flash
  - models/gemini-2.5-pro
  - models/gemini-2.0-flash-exp
  - models/gemini-2.0-flash
  - models/gemini-2.0-flash-001
  - models/gemini-2.0-flash-lite-001
  - models/gemini-2.0-flash-lite
  - models/gemini-2.0-flash-lite-preview-02-05
  - models/gemini-2.0-flash-lite-preview
  - models/gemini-exp-1206
  - models/gemini-2.5-flash-preview-tts
  - models/gemini-2.5-pro-preview-tts
  - models/gemini-flash-latest
  - models/gemini-flash-lite-latest
  - models/gemini-pro-latest
  - models/gemini-2.5-flash-lite
  - models/gemini-2.5-flash-image-preview
  - models/gemini-2.5-flash-image
  - models/gemini-2.5-flash-preview-09-2025
  - models/gemini-2.5-flash-lite-preview-09-2025
  - models/gemini-3-pro-preview
  - models/gemini-3-flash-preview
  - models/gemini-3-pro-image-preview
  - models/gemini-robotics-er-1.5-preview
  - models/gemini-2.5-computer-use-preview-10-2025
  -

# Semantic Search - OHS Legislation (Turkish)

Multi-AI Provider System with OpenAI GPT-4, Google Gemini, and Response Length Control

### Import the Needed Packages

In [71]:
import warnings
warnings.filterwarnings('ignore')

In [72]:
import sys

In [73]:
sys.path.append('../../../../DLAIUtils')

In [95]:
#!pip install python-dotenv
#!pip install datasets==2.15.0
#!pip install PyPDF2
#!pip install openai
#!pip install google-generativeai
!pip install -U google-generativeai

In [75]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from DLAIUtils import Utils
import os
import time
import torch
from dotenv import load_dotenv
import PyPDF2
from openai import OpenAI
import google.generativeai as genai

# Load environment variables from .env file
load_dotenv()

True

In [76]:
from tqdm.auto import tqdm

### Load the OHS Legislation PDF Dataset

In [77]:
# Load ISG Mevzuat PDF
pdf_path = 'data/isg_mevzuat.pdf'

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file"""
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
    return text

# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)
print(f"Total characters extracted: {len(pdf_text)}")
print(f"First 500 characters:\n{pdf_text[:500]}")

Total characters extracted: 234007
First 500 characters:
Bu doküman Global Belgelendirme Hizmetleri A.Ş. tarafından 01.06.2021 tarihinde oluşturulmuştur.  
  
 
 
 
 
 
 
 
 
4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  ...........................................................................................................  4 
Alt İşverenlik Yönetmeliği  .......................................................................................


In [78]:
# Improved text chunking to preserve complete sentences
def split_text_into_chunks(text, chunk_size=800, overlap=200):
    """Split text into overlapping chunks at sentence boundaries"""
    import re
    
    # Split into sentences (Turkish-aware)
    sentence_endings = r'[.!?]\s+'
    sentences = re.split(sentence_endings, text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        # If adding this sentence exceeds chunk_size and we have content, save chunk
        if len(current_chunk) + len(sentence) > chunk_size and current_chunk:
            chunks.append(current_chunk.strip())
            # Keep overlap: start new chunk with last part of previous chunk
            words = current_chunk.split()
            overlap_words = words[-int(len(words) * 0.2):] if len(words) > 5 else []
            current_chunk = ' '.join(overlap_words) + ' ' + sentence
        else:
            current_chunk += ' ' + sentence
    
    # Add the last chunk
    if current_chunk.strip():
        chunks.append(current_chunk.strip())
    
    return chunks

# Create chunks from PDF text with better sentence preservation
text_chunks = split_text_into_chunks(pdf_text, chunk_size=800, overlap=200)
print(f"Number of text chunks: {len(text_chunks)}")
print(f"\nFirst 3 chunks:")
for i, chunk in enumerate(text_chunks[:3]):
    print(f"\n--- Chunk {i+1} (length: {len(chunk)}) ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)

Number of text chunks: 405

First 3 chunks:

--- Chunk 1 (length: 699) ---
Bu doküman Global Belgelendirme Hizmetleri A.Ş tarafından 01.06.2021 tarihinde oluşturulmuştur 4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  .....................................................

--- Chunk 2 (length: 779) ---
11 Çocuk ve Genç İşçilerin Çalıştırılma Usul ve Esasları Hakkında Yönetmelik ............................ 12 
Gebe veya Emziren Kadınların Çalıştırılma Şartlarıyla İlgili Yönetmelik  .................................. 13 
Haftalık İş Günlerine Bölünemeyen Çalışma Süreleri Yönetmeliği  .................

--- Chunk 3 (length: 751) ---
Çalışma Yönetmeliği .............................. 17 Kadın Ça lışanların Gece Postalarında Çalıştırılma Koşulları Hakkında Yönetmelik .................. 18 
Postalar Halinde İşçi Çalıştırılarak Yürütülen İşlerde Çalışmalara İlişkin Yönetmelik  ..........

In [79]:
# Clean and prepare ISG mevzuat chunks
def clean_text(text):
    # Replace problematic characters with ASCII equivalents
    text = text.replace('–', '-')  # en dash
    text = text.replace('—', '-')  # em dash
    text = text.replace(''', "'")  # left single quotation mark
    text = text.replace(''', "'")  # right single quotation mark
    text = text.replace('"', '"')  # left double quotation mark
    text = text.replace('"', '"')  # right double quotation mark
    # Keep Turkish characters but remove other problematic Unicode
    # Don't encode to ASCII for Turkish text
    return text

# Clean the chunks
cleaned_chunks = [clean_text(chunk) for chunk in text_chunks if len(chunk.strip()) > 50]

print(f'Number of cleaned chunks: {len(cleaned_chunks)}')
print('\nFirst 2 chunks:')
for i, chunk in enumerate(cleaned_chunks[:2]):
    print(f"\n--- Chunk {i+1} ---")
    print(chunk[:300] + "..." if len(chunk) > 300 else chunk)

Number of cleaned chunks: 405

First 2 chunks:

--- Chunk 1 ---
Bu doküman Global Belgelendirme Hizmetleri A.Ş tarafından 01.06.2021 tarihinde oluşturulmuştur 4857 SAYILI İŞ KANUNU  
6331 SAYILI İŞ SAĞLIĞI VE GÜVENLİĞİ 
KANUNU  
BAĞLI DİĞER MEVZUAT  
 
 
 
 
 
 
 
 
 
 
 
1 
 İÇİNDEKİLER  
4857 SAYILI İŞ KANUNU  .....................................................

--- Chunk 2 ---
11 Çocuk ve Genç İşçilerin Çalıştırılma Usul ve Esasları Hakkında Yönetmelik ............................ 12 
Gebe veya Emziren Kadınların Çalıştırılma Şartlarıyla İlgili Yönetmelik  .................................. 13 
Haftalık İş Günlerine Bölünemeyen Çalışma Süreleri Yönetmeliği  .................


### Check cuda and Setup the model

**Note**: "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this course, we are using CPUs. So, you might notice some code cells taking a little longer to run.

We are using *all-MiniLM-L6-v2* sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [80]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [81]:
query = 'işçi sağlığı ve güvenliği nedir?'
xq = model.encode(query)
xq.shape

(384,)

In [82]:
utils = Utils()

### Setup Pinecone

In [83]:
# Load API keys from .env file
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Verify Pinecone API key is loaded
if not PINECONE_API_KEY or PINECONE_API_KEY == 'your_pinecone_api_key_here':
    raise ValueError("Please set PINECONE_API_KEY in .env file")

# Configure Google Gemini
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("✓ API keys loaded from .env file (OpenAI + Google Gemini)")
else:
    print("✓ API keys loaded from .env file (OpenAI only)")

✓ API keys loaded from .env file (OpenAI + Google Gemini)


In [84]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = utils.create_dlai_index_name('isg')

# Check and delete existing index if present
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    print(f"Deleting existing index: {INDEX_NAME}")
    pinecone.delete_index(INDEX_NAME)
    time.sleep(5)  # Wait for deletion to complete

print(f"Creating new index: {INDEX_NAME}")
pinecone.create_index(
    name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-east-1')
)

# Wait for index to be ready
print("Waiting for index to be ready...")
time.sleep(10)

# Connect to index
index = pinecone.Index(INDEX_NAME)
print(f"✓ Index ready: {index}")
print(f"✓ Index stats: {index.describe_index_stats()}")

Deleting existing index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Creating new index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Creating new index: isg-86hbriq-qif5lhm6p-ygourxdueztik7oaqa
Waiting for index to be ready...
Waiting for index to be ready...
✓ Index ready: <pinecone.db_data.index.Index object at 0x32e05f3d0>
✓ Index ready: <pinecone.db_data.index.Index object at 0x32e05f3d0>
✓ Index stats: {'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '150',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 14 Jan 2026 18:28:45 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '30',
                                    'x-pinecone-request-id': '8542425201013363533',
                                    'x-pinecone-req

### Create Embeddings and Upsert to Pinecone

In [85]:
batch_size = 100
# Use all chunks from ISG mevzuat
isg_chunks = cleaned_chunks

# Create a mapping to store the actual text (for later retrieval)
text_mapping = {i: text for i, text in enumerate(isg_chunks)}

print(f"Total chunks to process: {len(isg_chunks)}")

for i in tqdm(range(0, len(isg_chunks), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(isg_chunks))
    # create IDs batch
    ids = [f"isg_{x}" for x in range(i, i_end)]
    # create embeddings
    xc = model.encode(isg_chunks[i:i_end])
    # create metadata batch - store only chunk_id to avoid encoding issues
    metadatas = [{'chunk_id': idx} for idx in range(i, i_end)]
    # create records list for upsert with proper format
    records = [
        {
            'id': id_,
            'values': values.tolist(),
            'metadata': metadata
        }
        for id_, values, metadata in zip(ids, xc, metadatas)
    ]
    # upsert to Pinecone
    index.upsert(vectors=records)

print(f"\n✓ Successfully uploaded {len(isg_chunks)} ISG mevzuat chunks to Pinecone")
print(f"Text mapping created with {len(text_mapping)} entries")

Total chunks to process: 405


  0%|          | 0/5 [00:00<?, ?it/s]


✓ Successfully uploaded 405 ISG mevzuat chunks to Pinecone
Text mapping created with 405 entries


In [86]:
index.describe_index_stats()

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '185',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 14 Jan 2026 18:28:55 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '33',
                                    'x-pinecone-request-id': '495632367605095937',
                                    'x-pinecone-request-latency-ms': '32',
                                    'x-pinecone-response-duration-ms': '34'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 405}},
 'storageFullness': 0.0,
 'total_vector_count': 405,
 'vector_type': 'dense'}

### Run Your Queries - Multi-AI Provider Support

In [96]:
# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Enhanced query function with multiple AI providers and response length control
def run_query(query, use_ai=True, ai_provider="openai", use_reasoning=False, max_length="normal", top_k=5):
    """
    Query ISG legislation with semantic search and optional AI enhancement
    
    Args:
        query: The question to ask (in Turkish)
        use_ai: Whether to use AI for response synthesis (default: True)
        ai_provider: "openai", "gemini", or "none" (default: "openai")
        use_reasoning: Use advanced reasoning models (default: False)
        max_length: "short" (3-4 sentences), "medium" (5-7 sentences), or "normal" (detailed)
        top_k: Number of relevant chunks to retrieve (default: 5)
    """
    embedding = model.encode(query).tolist()
    results = index.query(top_k=top_k, vector=embedding, include_metadata=True, include_values=False)
    
    if not use_ai or ai_provider == "none":
        # Simple mode: just print the chunks
        print("🔍 Most Relevant Results:\n")
        for i, result in enumerate(results['matches'], 1):
            chunk_id = result['metadata']['chunk_id']
            text = text_mapping.get(chunk_id, "Text not found")
            print(f"{i}. [Score: {round(result['score'], 2)}]\n{text}\n")
    else:
        # AI mode: use AI to synthesize a better answer
        if ai_provider == "gemini":
            if use_reasoning:
                print("🧠 Deep Analysis with Gemini 1.5 Pro:\n")
            else:
                print("🤖 Enhanced Response with Gemini 1.5 Flash:\n")
        else:  # openai
            if use_reasoning:
                print("🧠 Deep Analysis with GPT-4 (o1 Reasoning):\n")
            else:
                print("🤖 Enhanced Response with GPT-4 Turbo:\n")
        
        # Gather relevant chunks
        relevant_texts = []
        for result in results['matches']:
            chunk_id = result['metadata']['chunk_id']
            text = text_mapping.get(chunk_id, "")
            if text:
                relevant_texts.append(text)
        
        context = "\n\n".join(relevant_texts)
        
        # Length-aware instructions
        if max_length == "short":
            length_instruction = "VERY IMPORTANT: Limit your response to maximum 3-4 sentences. Only mention the most important points."
            max_tokens = 300
        elif max_length == "medium":
            length_instruction = "Keep your response concise (5-7 sentences)."
            max_tokens = 500
        else:  # normal
            length_instruction = "Provide a detailed but concise response."
            max_tokens = 2048
        
        system_prompt = f"""You are an expert on OHS (Occupational Health and Safety) legislation. 
Answer questions based ONLY on the provided legislation texts. 
Provide answers in proper Turkish, in a bullet-point format, and reference relevant articles.
{length_instruction}"""

        user_prompt = f"""Answer the question based on the following OHS legislation texts.

Question: {query}

Legislation Texts:
{context}

Please:
1. Answer the question fully and clearly
2. Use ONLY the information from the provided legislation texts
3. Provide the answer in proper Turkish, in bullet-point format
4. Reference relevant articles
5. Give practical examples
6. Highlight important points
{length_instruction}"""

        # Call AI provider
        if ai_provider == "gemini":
            if use_reasoning:
                # Use Gemini 1.5 Pro with extended thinking (FREE!)
                gemini_model = genai.GenerativeModel('gemini-1.5-flash')
                response = gemini_model.generate_content(
                    f"{system_prompt}\n\n{user_prompt}",
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': max_tokens
                    }
                )
                print("💭 Used Gemini 1.5 Pro (with reasoning)\n")
                print(response.text)
            else:
                # Use Gemini 1.5 Flash (fastest, FREE!)
                gemini_model = genai.GenerativeModel('gemini-1.5-flash')
                response = gemini_model.generate_content(
                    f"{system_prompt}\n\n{user_prompt}",
                    generation_config={
                        'temperature': 0.3,
                        'max_output_tokens': max_tokens
                    }
                )
                print(response.text)
        else:  # openai
            if use_reasoning:
                # Use o1 model for deep reasoning
                response = openai_client.chat.completions.create(
                    model="o1-preview",
                    messages=[
                        {"role": "user", "content": f"{system_prompt}\n\n{user_prompt}"}
                    ]
                )
                print("💭 Used O1 Reasoning Model (with internal reasoning)\n")
                print(response.choices[0].message.content)
            else:
                # Use GPT-4 for standard response
                response = openai_client.chat.completions.create(
                    model="gpt-4-turbo-preview",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt}
                    ],
                    temperature=0.3,
                    max_tokens=max_tokens
                )
                print(response.choices[0].message.content)
        
        print("\n" + "="*80)
        print(f"📚 Source Texts (showing top {min(3, len(relevant_texts))}):")
        for i, text in enumerate(relevant_texts[:3], 1):
            print(f"\n{i}. {text[:200]}...")
    
    return results

In [97]:
# GPT-4 Turbo - Normal length response
run_query('işçi sağlığı ve güvenliği ile ilgili temel kurallar nelerdir?', 
          use_ai=True, ai_provider="openai", max_length="normal")

🤖 Enhanced Response with GPT-4 Turbo:

İş sağlığı ve güvenliği ile ilgili temel kurallar, yukarıda verilen mevzuat metinlerine dayanarak, aşağıdaki gibidir:

- **Koruyucu ve Önleyici Hizmetler:**
  - İşverenin, işyerindeki sağlık ve güvenlik risklerini önlemek üzere koruyucu hizmetleri yürütmek üzere işyerinden bir veya birden fazla kişiyi görevlendirmesi gerekmektedir.
  - Sağlık ve güvenlikle görevli kişiler, bu görevleri nedeniyle dezavantajlı duruma düşmezler ve görevlerini yerine getirebilmeleri için yeterli zaman ayrılmalıdır.

- **Çalışanların Sorumlulukları:**
  - Çalışanlar, işyeri hekimi, iş güvenliği uzmanı veya işveren tarafından verilen iş sağlığı ve güvenliğiyle ilgili mevzuata uygun talimatlara uymalıdır.
  - İş sağlığı ve güvenliği hizmetlerini yerine getirmek üzere işveren tarafından görevlendirilen kişi veya OSGB'lerin yapacağı çalışmalarda iş birliği yapmalıdır.

- **Tehlikelerin Tanımlanması:**
  - Tehlikelerin tanımlanması sırasında, işyeri bina ve eklentileri, yür

QueryResponse(matches=[{'id': 'isg_14',
 'metadata': {'chunk_id': 14},
 'score': 0.697583258,
 'values': []}, {'id': 'isg_10',
 'metadata': {'chunk_id': 10},
 'score': 0.695350707,
 'values': []}, {'id': 'isg_376',
 'metadata': {'chunk_id': 376},
 'score': 0.694004118,
 'values': []}, {'id': 'isg_212',
 'metadata': {'chunk_id': 212},
 'score': 0.687288821,
 'values': []}, {'id': 'isg_231',
 'metadata': {'chunk_id': 231},
 'score': 0.68340975,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 18:37:06 GMT', 'content-type': 'application/json', 'content-length': '445', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '61', 'x-pinecone-request-id': '3087970536682461823', 'x-envoy-upstream-service-time': '62', 'x-pinecone-response-duration-ms': '63', 'grpc-status': '0', 'server': 'envoy'}})

In [98]:
# Gemini 1.5 Pro - Deep reasoning (FREE!)
query = 'iş yerinde güvenlik önlemleri nelerdir?'
run_query(query, use_ai=True, ai_provider="gemini", use_reasoning=True, max_length="normal")

🧠 Deep Analysis with Gemini 1.5 Pro:



NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [94]:
# Gemini 1.5 Flash - Short response (3-4 sentences, FASTEST!)
query = 'işveren sorumlulukları nelerdir?'
run_query(query, use_ai=True, ai_provider="gemini", max_length="short")

🤖 Enhanced Response with Gemini 1.5 Flash:



NotFound: 404 models/gemini-1.5-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

In [ ]:
# GPT-4 - Medium response (5-7 sentences)
query = 'çalışanların eğitim yükümlülükleri nelerdir?'
run_query(query, use_ai=True, ai_provider="openai", max_length="medium")

🤖 Enhanced Response with GPT-4 Turbo:

Çalışanların eğitim yükümlülükleri, sağlanan mevzuat metinlerine göre şu şekilde özetlenebilir:

- Çalışanlar, işe alınmadan önce, Ek-1 çizelgesinde yer alan işlerde çalışacaklarsa, mesleki eğitime tabi tutulmaları zorunludur. Bu, özellikle tehlikeli ve çok tehlikeli sınıfta yer alan işler için geçerlidir ve çalışanların bu eğitimleri tamamlamaları gerekmektedir.

- Çalışanların, iş sağlığı ve güvenliği bakımından, ilgili mevzuatın öngördüğü esaslara ve işverenin bu esaslara uygun olarak vereceği emir ve talimatlara uymaları gerekmektedir. Bu, genel iş güvenliği kurallarının yanı sıra, özel olarak verilen güvenlik talimatlarına da uymayı içerir.

- Çalışanlar, aldıkları eğitim ve işverenin talimatları doğrultusunda, kendilerinin ve hareketlerinden veya yaptıkları işten etkilenen diğer çalışanların sağlık ve güvenliklerini tehlikeye düşürmemekle yükümlüdürler. Bu, güvenlik donanımlarını doğru kullanmayı, keyfi olarak çıkarmamayı ve değiştirmemeyi d

QueryResponse(matches=[{'id': 'isg_311',
 'metadata': {'chunk_id': 311},
 'score': 0.731275499,
 'values': []}, {'id': 'isg_13',
 'metadata': {'chunk_id': 13},
 'score': 0.679422438,
 'values': []}, {'id': 'isg_298',
 'metadata': {'chunk_id': 298},
 'score': 0.675013602,
 'values': []}, {'id': 'isg_128',
 'metadata': {'chunk_id': 128},
 'score': 0.670695364,
 'values': []}, {'id': 'isg_10',
 'metadata': {'chunk_id': 10},
 'score': 0.668191969,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 18:22:03 GMT', 'content-type': 'application/json', 'content-length': '446', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '7', 'x-pinecone-request-id': '5333944746032222272', 'x-envoy-upstream-service-time': '7', 'x-pinecone-response-duration-ms': '9', 'grpc-status': '0', 'server': 'envoy'}})

### AI Provider Comparison & Response Length Control

**3 AI Providers:**
1. **No AI** (`use_ai=False`): Raw chunks only - instant results
2. **OpenAI GPT-4** (`ai_provider="openai"`): High quality, paid, slower (15-25 sec)
3. **Google Gemini** (`ai_provider="gemini"`): Fast, FREE with generous limits! (5-10 sec)

**Reasoning Models:**
- **O1-Preview** (OpenAI): Deep reasoning, slower (20-30 sec), more expensive
- **Gemini 1.5 Pro** (`use_reasoning=True`): Advanced reasoning, FREE, faster (8-15 sec)

**Response Length:**
- `max_length="short"`: 3-4 sentences (quick answers, 5-8 sec)
- `max_length="medium"`: 5-7 sentences (balanced, 8-12 sec)
- `max_length="normal"`: Detailed response (default, 10-20 sec)

**Speed & Cost Comparison:**
- **Gemini 1.5 Flash**: FREE, fastest (5-10 sec), 1,500 requests/day
- **Gemini 1.5 Pro**: FREE, fast (8-15 sec), better reasoning
- **GPT-4 Turbo**: $10/1M input tokens, slower (15-25 sec)
- **O1-Preview**: $15/1M input tokens, slowest (20-30 sec)

**Recommendation:** Use Gemini for speed and cost-effectiveness!

In [ ]:
# Complex question with Gemini 1.5 Pro Reasoning (FREE!)
run_query(
    'işveren ve çalışanların iş kazalarını önlemedeki sorumlulukları nelerdir ve bunlar nasıl denetlenir?',
    use_ai=True,
    ai_provider="gemini",
    use_reasoning=True,
    max_length="normal"
)

🧠 Deep Analysis with Gemini 2.0 Flash Thinking:



NotFound: 404 models/gemini-2.0-flash-thinking-exp-01-21 is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

### Side-by-Side Comparison: OpenAI vs Gemini vs No AI

In [ ]:
print("\n" + "=" * 80)
print("2. GEMINI 1.5 FLASH (FREE, Short Response - Fastest!):")
print("=" * 80)
run_query('iş kazası durumunda ne yapılmalıdır?', use_ai=True, ai_provider="gemini", max_length="short")


2. GEMINI 2.0 FLASH (FREE, Short Response):
🤖 Enhanced Response with Gemini 2.0 Flash:



ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-exp
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash-exp
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash-exp
Please retry in 49.534143906s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-Fre

In [ ]:
print("\n" + "=" * 80)
print("3. GPT-4 TURBO (Paid, Medium Response - Slower):")
print("=" * 80)
run_query('iş kazası durumunda ne yapılmalıdır?', use_ai=True, ai_provider="openai", max_length="medium")


3. GPT-4 TURBO (Paid, Medium Response):
🤖 Enhanced Response with GPT-4 Turbo:

İş kazası durumunda yapılması gerekenler:

- İlk olarak, iş kazası meydana geldiğinde derhal ilk yardım müdahalesi yapılmalıdır. (İlkyardım Yönetmeliği)
- Kazanın işverene ve gerekirse ilgili diğer yetkili kurumlara bildirilmesi gerekmektedir. İşveren, kazayla ilgili tüm detayları kaydetmeli ve gerekli raporlamaları yapmalıdır. (İş Sağlığı ve Güvenliği Yönetmeliği)
- İşveren, iş kazası sonucu yaralanan çalışanın sağlık kuruluşuna sevk edilmesini sağlamalı ve gerekli tıbbi müdahalenin yapılmasını garanti altına almalıdır.
- İş kazası sonrası, işyerindeki iş sağlığı ve güvenliği faaliyetlerine ilişkin kayıtların güncellenmesi ve kazanın analizi yapılmalıdır. Bu analiz, benzer kazaların önlenmesi için alınacak tedbirlerin belirlenmesinde kullanılmalıdır. (İşverenin sağlık ve güvenlik kayıtları ve onaylı deftere ilişkin yükümlülükleri)
- Çalışanlar, kaza ile ilgili olarak sağlık ve güvenliklerini etkileyebilece

QueryResponse(matches=[{'id': 'isg_10',
 'metadata': {'chunk_id': 10},
 'score': 0.637305319,
 'values': []}, {'id': 'isg_46',
 'metadata': {'chunk_id': 46},
 'score': 0.612169325,
 'values': []}, {'id': 'isg_211',
 'metadata': {'chunk_id': 211},
 'score': 0.608727455,
 'values': []}, {'id': 'isg_164',
 'metadata': {'chunk_id': 164},
 'score': 0.606282234,
 'values': []}, {'id': 'isg_14',
 'metadata': {'chunk_id': 14},
 'score': 0.599728644,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 18:23:19 GMT', 'content-type': 'application/json', 'content-length': '444', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '5', 'x-pinecone-request-latency-ms': '5', 'x-pinecone-request-id': '8408914194730189559', 'x-envoy-upstream-service-time': '6', 'x-pinecone-response-duration-ms': '8', 'grpc-status': '0', 'server': 'envoy'}})

In [ ]:
print("=" * 80)
print("1. NO AI (Raw Chunks - Instant Results):")
print("=" * 80)
run_query('iş kazası durumunda ne yapılmalıdır?', use_ai=False)

1. NO AI (Raw Chunks):
🔍 Most Relevant Results:

1. [Score: 0.64]
76 Tehlikeli ve Çok Tehlikeli Sınıfta Yer Alan İşlerde Çalıştırılacakların Mesleki Eğitimler i Hk Yön 78 
Tozla Mücadele Yönetmeliği  ........................................ .......................................................... 78 
Ulusal İş Sağlığı ve Güvenliği Konseyi Yönetmeliği  .................................................................. 79 
3 
 Yapı İşlerinde İş Sağlığı ve Güvenliği Yönetmeliği  ................... ........................ ....................... 79 
Asansör İşletme, Bakım ve Periyodik Kontrol Yönetmeliği  .................................................. ... 80 
Asansör Periyodik Kontrol Yönetmeliği  ................................................. .................................

2. [Score: 0.61]
süre günlük çalışma süresinden sayılır İşçi özlük dosyası: İşveren çalıştırdığı her işçi için bir özlük dosyası düzenler 11 
 4857 SAYILI İŞ KANUNU ’NA BAĞLI YÖNETMELİKLER  
 
ALT İŞVEREN

QueryResponse(matches=[{'id': 'isg_10',
 'metadata': {'chunk_id': 10},
 'score': 0.637305319,
 'values': []}, {'id': 'isg_46',
 'metadata': {'chunk_id': 46},
 'score': 0.612169325,
 'values': []}, {'id': 'isg_211',
 'metadata': {'chunk_id': 211},
 'score': 0.608727455,
 'values': []}, {'id': 'isg_164',
 'metadata': {'chunk_id': 164},
 'score': 0.606282234,
 'values': []}, {'id': 'isg_14',
 'metadata': {'chunk_id': 14},
 'score': 0.599728644,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Wed, 14 Jan 2026 18:05:03 GMT', 'content-type': 'application/json', 'content-length': '444', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '10', 'x-pinecone-request-latency-ms': '41', 'x-pinecone-request-id': '313358787445261396', 'x-envoy-upstream-service-time': '42', 'x-pinecone-response-duration-ms': '43', 'grpc-status': '0', 'server': 'envoy'}})